In [1]:
!pip install osmnx
!pip install matplotlib
!pip install geopandas
!pip install folium

In [2]:
import folium
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd

In [4]:
def is_bike_friendly(edge):
    return (
        edge.get('bicycle', 'no') in ('yes', 'designated', 'permissive') or
        edge.get('highway') == 'cycleway' or
        'cycleway' in edge
    )

# Define the place and create a graph from OSM data
place_name = "Hanover"
G = ox.graph_from_place(place_name, network_type='all')

# Get bus stops and bike-friendly paths
tags = {'highway': 'bus_stop'}
bus_stops = ox.geometries_from_place(place_name, tags=tags)

# Get the edges and nodes from the graph
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
bike_friendly_edges = edges[edges.apply(is_bike_friendly, axis=1)]
nodes = ox.graph_to_gdfs(G, nodes=True, edges=False)

# Convert to the same CRS and create buffers
bus_stops = bus_stops.to_crs('EPSG:3395')
bus_stops_buffer = bus_stops.buffer(15)  # 100 meters buffer for bus stops
bike_friendly_edges = bike_friendly_edges.to_crs('EPSG:3395')
bike_friendly_buffer = bike_friendly_edges.buffer(15)  # 20 meters buffer for bike paths
nodes = nodes.to_crs('EPSG:3395')

# Find nodes near bus stops and bike paths
near_bus_stops = nodes[nodes.geometry.apply(lambda x: bus_stops_buffer.contains(x).any())]
near_bike_paths = nodes[nodes.geometry.apply(lambda x: bike_friendly_buffer.contains(x).any())]
near_both = near_bus_stops[near_bus_stops.index.isin(near_bike_paths.index)]

# Convert back to the original CRS for plotting
near_both = near_both.to_crs('EPSG:4326')
bike_friendly_edges = bike_friendly_edges.to_crs('EPSG:4326')
bus_stops = bus_stops.to_crs('EPSG:4326')

# Create a base map with the Stamen Terrain tiles and attribution
map = folium.Map(location=[52.3705, 9.7332], zoom_start=13)

# Add bike-friendly paths to the map
for _, row in bike_friendly_edges.iterrows():
    line = folium.PolyLine(locations=[(lat, lon) for lon, lat in row['geometry'].coords], color='green', weight=5)
    map.add_child(line)
# Add intersections near both bus stops and bike paths
for _, row in near_both.iterrows():
    folium.CircleMarker(location=(row.geometry.y, row.geometry.x), radius=7, color='red', fill=True).add_to(map)

number_of_danger_points = len(near_both)
print(number_of_danger_points)
for _, row in near_both.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=7, 
        color='red', 
        fill=True,
        tooltip=f'Danger Point - Total: {number_of_danger_points}'
    ).add_to(map)
# Display the map
map

bike_friendly_edges = edges[edges.apply(is_bike_friendly, axis=1)]

# Determine the UTM zone and convert bike-friendly paths to this CRS
utm_crs = ox.projection.get_utm_crs(52.3705, 9.7332)
bike_friendly_edges_utm = bike_friendly_edges.to_crs(utm_crs)

# Calculate the total length in UTM CRS
total_length_meters = bike_friendly_edges_utm['geometry'].length.sum()
total_length_km = total_length_meters / 1000
print(f"Total length of bike-friendly paths: {total_length_km:.2f} km")

/var/folders/vz/b1smgzt13_dbs5c1rbj4m3dh0000gn/T/ipykernel_25147/3351815376.py:14: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  bus_stops = ox.geometries_from_place(place_name, tags=tags)


42


AttributeError: module 'osmnx.projection' has no attribute 'get_utm_crs'